<a href="https://colab.research.google.com/github/AmanPriyanshu/phishing_classifier/blob/master/phishing_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving phishing.csv to phishing.csv


In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import trange
import time
from sklearn.cluster import KMeans

In [0]:
data = pd.read_csv('phishing.csv')
features = data.columns[1:]
data = data.values
data = data.T[1:]
data = data.T

In [4]:
print(features)

Index(['UsingIP', 'LongURL', 'ShortURL', 'Symbol@', 'Redirecting//',
       'PrefixSuffix-', 'SubDomains', 'HTTPS', 'DomainRegLen', 'Favicon',
       'NonStdPort', 'HTTPSDomainURL', 'RequestURL', 'AnchorURL',
       'LinksInScriptTags', 'ServerFormHandler', 'InfoEmail', 'AbnormalURL',
       'WebsiteForwarding', 'StatusBarCust', 'DisableRightClick',
       'UsingPopupWindow', 'IframeRedirection', 'AgeofDomain', 'DNSRecording',
       'WebsiteTraffic', 'PageRank', 'GoogleIndex', 'LinksPointingToPage',
       'StatsReport', 'class'],
      dtype='object')


In [5]:
data[:5]

array([[ 1,  1,  1,  1,  1, -1,  0,  1, -1,  1,  1, -1,  1,  0, -1, -1,
         1,  1,  0,  1,  1,  1,  1, -1, -1,  0, -1,  1,  1,  1, -1],
       [ 1,  0,  1,  1,  1, -1, -1, -1, -1,  1,  1, -1,  1,  0, -1, -1,
        -1, -1,  0,  1,  1,  1,  1,  1, -1,  1, -1,  1,  0, -1, -1],
       [ 1,  0,  1,  1,  1, -1, -1, -1,  1,  1,  1, -1, -1,  0,  0, -1,
         1,  1,  0,  1,  1,  1,  1, -1, -1,  1, -1,  1, -1,  1, -1],
       [ 1,  0, -1,  1,  1, -1,  1,  1, -1,  1,  1,  1,  1,  0,  0, -1,
         1,  1,  0, -1,  1, -1,  1, -1, -1,  0, -1,  1,  1,  1,  1],
       [-1,  0, -1,  1, -1, -1,  1,  1, -1,  1,  1, -1,  1,  0,  0, -1,
        -1, -1,  0,  1,  1,  1,  1,  1,  1,  1, -1,  1, -1, -1,  1]])

X, Y splitter. Also creation of only normal cases and phishing cases as well as random

In [0]:
X = data.T[:-1]
X = X.T
Y = data.T[-1]
Y = np.array([0 if i==-1 else 1 for i in Y])

shuffled_indexes = np.array(range(X.shape[0]))
np.random.shuffle(shuffled_indexes)

X_random = X[shuffled_indexes]
Y_random = Y[shuffled_indexes]

X_train, X_test = X_random[:int(0.8*X_random.shape[0])], X_random[int(0.8*X_random.shape[0]):]
Y_train, Y_test = Y_random[:int(0.8*X_random.shape[0])], Y_random[int(0.8*X_random.shape[0]):]

normal_webistes = np.array([X[index] for index in range(Y.shape[0]) if Y[index] == 1])
phishing_webistes = np.array([X[index] for index in range(Y.shape[0]) if Y[index] == 0])

In [7]:
print(normal_webistes.shape)
print(phishing_webistes.shape)
print(X_random)

(6157, 30)
(4897, 30)
[[ 1 -1 -1 ... -1  0  1]
 [ 1 -1  1 ...  1  0  1]
 [ 1  1  1 ...  1  0  1]
 ...
 [-1 -1  1 ...  1  1  1]
 [ 1 -1  1 ...  1  0  1]
 [-1 -1 -1 ... -1  0  1]]


##SUPERVISED

LOGISTIC REGRESSION

In [8]:
model = tf.keras.models.Sequential([
                                      tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l1(l=0.001))
  ])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=100,verbose=1, validation_data=(X_test, Y_test))

Epoch 1/100
277/277 [==============================] - 1s 2ms/step - loss: 0.6043 - accuracy: 0.6943 - val_loss: 0.4832 - val_accuracy: 0.7956
Epoch 2/100
277/277 [==============================] - 0s 1ms/step - loss: 0.4143 - accuracy: 0.8477 - val_loss: 0.3800 - val_accuracy: 0.8661
Epoch 3/100
277/277 [==============================] - 0s 1ms/step - loss: 0.3348 - accuracy: 0.8987 - val_loss: 0.3254 - val_accuracy: 0.8991
Epoch 4/100
277/277 [==============================] - 0s 1ms/step - loss: 0.2905 - accuracy: 0.9162 - val_loss: 0.2939 - val_accuracy: 0.9014
Epoch 5/100
277/277 [==============================] - 0s 1ms/step - loss: 0.2635 - accuracy: 0.9215 - val_loss: 0.2737 - val_accuracy: 0.9046
Epoch 6/100
277/277 [==============================] - 0s 1ms/step - loss: 0.2457 - accuracy: 0.9258 - val_loss: 0.2603 - val_accuracy: 0.9068
Epoch 7/100
277/277 [==============================] - 0s 1ms/step - loss: 0.2335 - accuracy: 0.9264 - val_loss: 0.2511 - val_accuracy: 0.9104

In [9]:
score = model.evaluate(X_train, Y_train, verbose=0) 
print('Train:', 'loss:',score[0], 'accuracy:',score[1])
score = model.evaluate(X_test, Y_test, verbose=0) 
print('Test:', 'loss:',score[0], 'accuracy:',score[1])

Train: loss: 0.19315561652183533 accuracy: 0.9314712285995483
Test: loss: 0.21913272142410278 accuracy: 0.9181365966796875


ANN:

In [10]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(15, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(7, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l1(l=0.001))
  ])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=100,verbose=1, validation_data=(X_test, Y_test))

Epoch 1/100
277/277 [==============================] - 1s 2ms/step - loss: 0.5006 - accuracy: 0.7690 - val_loss: 0.3013 - val_accuracy: 0.8942
Epoch 2/100
277/277 [==============================] - 0s 2ms/step - loss: 0.2938 - accuracy: 0.8958 - val_loss: 0.2488 - val_accuracy: 0.9077
Epoch 3/100
277/277 [==============================] - 0s 2ms/step - loss: 0.2444 - accuracy: 0.9142 - val_loss: 0.2243 - val_accuracy: 0.9154
Epoch 4/100
277/277 [==============================] - 0s 2ms/step - loss: 0.2305 - accuracy: 0.9203 - val_loss: 0.2203 - val_accuracy: 0.9150
Epoch 5/100
277/277 [==============================] - 0s 2ms/step - loss: 0.2164 - accuracy: 0.9266 - val_loss: 0.2085 - val_accuracy: 0.9190
Epoch 6/100
277/277 [==============================] - 0s 2ms/step - loss: 0.2071 - accuracy: 0.9271 - val_loss: 0.1982 - val_accuracy: 0.9231
Epoch 7/100
277/277 [==============================] - 0s 2ms/step - loss: 0.2001 - accuracy: 0.9305 - val_loss: 0.1918 - val_accuracy: 0.9236

In [11]:
score = model.evaluate(X_train, Y_train, verbose=0) 
print('Train:', 'loss:',score[0], 'accuracy:',score[1])
score = model.evaluate(X_test, Y_test, verbose=0) 
print('Test:', 'loss:',score[0], 'accuracy:',score[1])

Train: loss: 0.08693104237318039 accuracy: 0.9685627222061157
Test: loss: 0.13639633357524872 accuracy: 0.9488918781280518


Naive Bayes

In [0]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
model = gnb.fit(X_train, Y_train)
y_pred = model.predict(X_test)

In [13]:
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (Y_test != y_pred).sum()))

Number of mislabeled points out of a total 2211 points : 896


##UNSUPERVISED

Auto-Encoders

In [14]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(15, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(7, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(7, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(15, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(30, activation='tanh')
])
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
history = model.fit(normal_webistes, normal_webistes, epochs=100,verbose=1)

Epoch 1/100
193/193 [==============================] - 0s 2ms/step - loss: 0.6641 - mse: 0.6641
Epoch 2/100
193/193 [==============================] - 0s 2ms/step - loss: 0.5313 - mse: 0.5313
Epoch 3/100
193/193 [==============================] - 0s 1ms/step - loss: 0.5090 - mse: 0.5090
Epoch 4/100
193/193 [==============================] - 0s 2ms/step - loss: 0.4816 - mse: 0.4816
Epoch 5/100
193/193 [==============================] - 0s 1ms/step - loss: 0.4560 - mse: 0.4560
Epoch 6/100
193/193 [==============================] - 0s 2ms/step - loss: 0.4395 - mse: 0.4395
Epoch 7/100
193/193 [==============================] - 0s 2ms/step - loss: 0.4291 - mse: 0.4291
Epoch 8/100
193/193 [==============================] - 0s 1ms/step - loss: 0.4250 - mse: 0.4250
Epoch 9/100
193/193 [==============================] - 0s 2ms/step - loss: 0.4204 - mse: 0.4204
Epoch 10/100
193/193 [==============================] - 0s 1ms/step - loss: 0.4138 - mse: 0.4138
Epoch 11/100
193/193 [=================

In [15]:
mse_train = []
acc = 0
for normal_i in trange(normal_webistes.shape[0]):
  normal = normal_webistes[normal_i]
  score = model.evaluate(np.reshape(np.array(normal), (1, 30)), np.reshape(np.array(normal), (1, 30)), verbose=0) 
  mse_train.append(score[1])
  if score[1] < (0.30717997736194236+0.4784793007228054)/2:
    acc += 1
mse_train = np.array(mse_train)
print(np.mean(mse_train))

time.sleep(1)

mse_test = []
for phishing_i in trange(phishing_webistes.shape[0]):
  phishing = phishing_webistes[phishing_i]
  score = model.evaluate(np.reshape(np.array(phishing), (1, 30)), np.reshape(np.array(phishing), (1, 30)), verbose=0) 
  mse_test.append(score[1])
  if score[1] > (0.29527+0.834192)/2:
    acc += 1
mse_test = np.array(mse_test)

time.sleep(1)

print(np.mean(mse_test))

print()
print('accuracy', acc/(phishing_webistes.shape[0] + normal_webistes.shape[0]))

100%|██████████| 6157/6157 [03:14<00:00, 31.59it/s]


0.32613883145178935


100%|██████████| 4897/4897 [02:34<00:00, 31.73it/s]


0.4956340214744597

accuracy 0.5232495024425547


K-means

In [0]:
def k_means(X, n_clusters):
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
  return kmeans.labels_

In [17]:
labels = k_means(X_train, 2)
print(1 - np.mean(np.square(Y_train - labels)))

0.452900599344114


##Feature Selection

In [0]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [19]:
model = LogisticRegression(solver='lbfgs')
rfe = RFE(model, 5)
fit = rfe.fit(X_random, Y_random)
print(fit)

RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=100,
                                 multi_class='auto', n_jobs=None, penalty='l2',
                                 random_state=None, solver='lbfgs', tol=0.0001,
                                 verbose=0, warm_start=False),
    n_features_to_select=5, step=1, verbose=0)


In [20]:
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)
best_features = [(name, rank) for name, rank in zip(features[:-1],fit.ranking_) if rank == 1]

Num Features: 5
Selected Features: [False False False False False  True False  True False False False False
 False  True  True  True False False False False False False False False
 False False False False False False]
Feature Ranking: [ 8 21  6 15 25  1  3  1 26 12 11 10 18  1  1  1 13 19  4 17 22 23 16 24
  9  2 20  5  7 14]


In [21]:
print(best_features)

[('PrefixSuffix-', 1), ('HTTPS', 1), ('AnchorURL', 1), ('LinksInScriptTags', 1), ('ServerFormHandler', 1)]


# Selected Data:

In [0]:
best_features = [i[0] for i in best_features] + ['class']

In [0]:
data = pd.read_csv('phishing.csv', usecols=best_features)
features = data.columns[:]
data = data.values
data = data.T[:]
data = data.T

In [24]:
print(features)
print(data[:5])

Index(['PrefixSuffix-', 'HTTPS', 'AnchorURL', 'LinksInScriptTags',
       'ServerFormHandler', 'class'],
      dtype='object')
[[-1  1  0 -1 -1 -1]
 [-1 -1  0 -1 -1 -1]
 [-1 -1  0  0 -1 -1]
 [-1  1  0  0 -1  1]
 [-1  1  0  0 -1  1]]


X, Y splitter. Also creation of only normal cases and phishing cases as well as random

In [0]:
X = data.T[:-1]
X = X.T
Y = data.T[-1]
Y = np.array([0 if i==-1 else 1 for i in Y])

shuffled_indexes = np.array(range(X.shape[0]))
np.random.shuffle(shuffled_indexes)

X_random = X[shuffled_indexes]
Y_random = Y[shuffled_indexes]

X_train, X_test = X_random[:int(0.8*X_random.shape[0])], X_random[int(0.8*X_random.shape[0]):]
Y_train, Y_test = Y_random[:int(0.8*X_random.shape[0])], Y_random[int(0.8*X_random.shape[0]):]

normal_webistes = np.array([X[index] for index in range(Y.shape[0]) if Y[index] == 1])
phishing_webistes = np.array([X[index] for index in range(Y.shape[0]) if Y[index] == 0])

In [26]:
print(normal_webistes.shape)
print(phishing_webistes.shape)
print(X_random)

(6157, 5)
(4897, 5)
[[ 1  1  0  0 -1]
 [-1  1  0  1 -1]
 [-1  0 -1 -1 -1]
 ...
 [-1  1  0 -1 -1]
 [-1  1  0  0  0]
 [ 1  1  0  1 -1]]


##SUPERVISED

LOGISTIC REGRESSION

In [27]:
model = tf.keras.models.Sequential([
                                      tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l1(l=0.001))
  ])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=100,verbose=1, validation_data=(X_test, Y_test))

Epoch 1/100
277/277 [==============================] - 0s 2ms/step - loss: 0.7768 - accuracy: 0.5316 - val_loss: 0.6593 - val_accuracy: 0.6436
Epoch 2/100
277/277 [==============================] - 0s 1ms/step - loss: 0.5657 - accuracy: 0.7693 - val_loss: 0.5106 - val_accuracy: 0.8476
Epoch 3/100
277/277 [==============================] - 0s 1ms/step - loss: 0.4556 - accuracy: 0.8706 - val_loss: 0.4285 - val_accuracy: 0.8761
Epoch 4/100
277/277 [==============================] - 0s 1ms/step - loss: 0.3921 - accuracy: 0.8854 - val_loss: 0.3781 - val_accuracy: 0.8801
Epoch 5/100
277/277 [==============================] - 0s 1ms/step - loss: 0.3520 - accuracy: 0.8871 - val_loss: 0.3451 - val_accuracy: 0.8801
Epoch 6/100
277/277 [==============================] - 0s 1ms/step - loss: 0.3254 - accuracy: 0.8875 - val_loss: 0.3225 - val_accuracy: 0.8801
Epoch 7/100
277/277 [==============================] - 0s 1ms/step - loss: 0.3064 - accuracy: 0.8904 - val_loss: 0.3058 - val_accuracy: 0.8969

In [28]:
score = model.evaluate(X_train, Y_train, verbose=0) 
print('Train:', 'loss:',score[0], 'accuracy:',score[1])
score = model.evaluate(X_test, Y_test, verbose=0) 
print('Test:', 'loss:',score[0], 'accuracy:',score[1])

Train: loss: 0.22749002277851105 accuracy: 0.9139432311058044
Test: loss: 0.23049600422382355 accuracy: 0.907281756401062


ANN

In [29]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(15, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(7, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l1(l=0.001))
  ])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=100,verbose=1, validation_data=(X_test, Y_test))

Epoch 1/100
277/277 [==============================] - 1s 2ms/step - loss: 0.5378 - accuracy: 0.7702 - val_loss: 0.3475 - val_accuracy: 0.9064
Epoch 2/100
277/277 [==============================] - 0s 2ms/step - loss: 0.3300 - accuracy: 0.8999 - val_loss: 0.2447 - val_accuracy: 0.9150
Epoch 3/100
277/277 [==============================] - 0s 2ms/step - loss: 0.2869 - accuracy: 0.9076 - val_loss: 0.2278 - val_accuracy: 0.9068
Epoch 4/100
277/277 [==============================] - 0s 2ms/step - loss: 0.2737 - accuracy: 0.9099 - val_loss: 0.2163 - val_accuracy: 0.9154
Epoch 5/100
277/277 [==============================] - 0s 2ms/step - loss: 0.2580 - accuracy: 0.9104 - val_loss: 0.2126 - val_accuracy: 0.9154
Epoch 6/100
277/277 [==============================] - 0s 2ms/step - loss: 0.2416 - accuracy: 0.9129 - val_loss: 0.2056 - val_accuracy: 0.9168
Epoch 7/100
277/277 [==============================] - 0s 2ms/step - loss: 0.2366 - accuracy: 0.9116 - val_loss: 0.2044 - val_accuracy: 0.9168

In [30]:
score = model.evaluate(X_train, Y_train, verbose=0) 
print('Train:', 'loss:',score[0], 'accuracy:',score[1])
score = model.evaluate(X_test, Y_test, verbose=0) 
print('Test:', 'loss:',score[0], 'accuracy:',score[1])

Train: loss: 0.18863289058208466 accuracy: 0.9262693524360657
Test: loss: 0.19344502687454224 accuracy: 0.9176843166351318


Naive Bayes

In [0]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
model = gnb.fit(X_train, Y_train)
y_pred = model.predict(X_test)

In [32]:
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (Y_test != y_pred).sum()))

Number of mislabeled points out of a total 2211 points : 931


## Unsupervised

AutoEncoders

In [33]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(5, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(3, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(2, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(3, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(5, activation='tanh')
])
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
history = model.fit(normal_webistes, normal_webistes, epochs=100,verbose=1)

Epoch 1/100
193/193 [==============================] - 0s 1ms/step - loss: 0.6992 - mse: 0.6992
Epoch 2/100
193/193 [==============================] - 0s 1ms/step - loss: 0.5748 - mse: 0.5748
Epoch 3/100
193/193 [==============================] - 0s 1ms/step - loss: 0.5371 - mse: 0.5371
Epoch 4/100
193/193 [==============================] - 0s 1ms/step - loss: 0.5282 - mse: 0.5282
Epoch 5/100
193/193 [==============================] - 0s 1ms/step - loss: 0.5226 - mse: 0.5226
Epoch 6/100
193/193 [==============================] - 0s 1ms/step - loss: 0.5204 - mse: 0.5204
Epoch 7/100
193/193 [==============================] - 0s 1ms/step - loss: 0.5183 - mse: 0.5183
Epoch 8/100
193/193 [==============================] - 0s 1ms/step - loss: 0.5159 - mse: 0.5159
Epoch 9/100
193/193 [==============================] - 0s 1ms/step - loss: 0.5092 - mse: 0.5092
Epoch 10/100
193/193 [==============================] - 0s 1ms/step - loss: 0.4636 - mse: 0.4636
Epoch 11/100
193/193 [=================

In [34]:
mse_train = []
acc = 0
for normal_i in trange(normal_webistes.shape[0]):
  normal = normal_webistes[normal_i]
  score = model.evaluate(np.reshape(np.array(normal), (1, 5)), np.reshape(np.array(normal), (1, 5)), verbose=0) 
  mse_train.append(score[1])
  if score[1] < (0.29527+0.834192)/2:
    acc += 1
mse_train = np.array(mse_train)
print(np.mean(mse_train))

time.sleep(1)

mse_test = []
for phishing_i in trange(phishing_webistes.shape[0]):
  phishing = phishing_webistes[phishing_i]
  score = model.evaluate(np.reshape(np.array(phishing), (1, 5)), np.reshape(np.array(phishing), (1, 5)), verbose=0) 
  mse_test.append(score[1])
  if score[1] > (0.29527+0.834192)/2:
    acc += 1
mse_test = np.array(mse_test)

time.sleep(1)

print(np.mean(mse_test))

print()
print('Accuracy',acc/(phishing_webistes.shape[0] + normal_webistes.shape[0]))

100%|██████████| 6157/6157 [03:15<00:00, 31.54it/s]


0.344087118494379


100%|██████████| 4897/4897 [02:36<00:00, 31.27it/s]


0.8766558148120529

Accuracy 0.809842590917315


K Means

In [0]:
def k_means(X, n_clusters):
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
  return kmeans.labels_

In [36]:
labels = k_means(X_train, 2)
print(1 - np.mean(np.square(Y_train - labels)))

0.88917788080968
